In [41]:
from dotenv import load_dotenv
load_dotenv()

True

In [42]:
from langgraph.graph import StateGraph,END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, AIMessage, HumanMessage, ToolMessage, SystemMessage
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.sqlite import SqliteSaver

# Manually enter context manager
memory_context = SqliteSaver.from_conn_string(":memory:")
memory = memory_context.__enter__()

In [43]:
from uuid import uuid4

def reduce_message(left:list[AnyMessage],right:list[AnyMessage]):
    for message in right:
        if not message.id:
            message.id=str(uuid4())
    
    merged=left.copy()
    for message in right:
        for i,existing in enumerate(merged):
            if existing.id==message.id:
                merged[i]=message
                break
        else:
            merged.append(message)
    return merged

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage],reduce_message]

In [44]:
tool=TavilySearchResults(max_results=2)

class Agent:
    def __init__(self,model,tools,checkpointer,system=""):
        self.system=system
        graph=StateGraph(AgentState)
        graph.add_node("llm",self.call_groq)
        graph.add_node("action",self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.action_exists,
            {True:"action",False:END}
        )
        graph.add_edge("action","llm")
        graph.set_entry_point("llm")
        self.graph=graph.compile(checkpointer=checkpointer,interrupt_before=['action'])
        self.tools={t.name:t for t in tools}
        self.model=model.bind_tools(tools)
        
    def action_exists(self,state:AgentState):
        result=state['messages'][-1]
        return len(result.tool_calls)>0
        
        
    def call_groq(self,state:AgentState):
        messages=state["messages"]
        if self.system:
            messages=[SystemMessage(content=self.system)] + messages
        message=self.model.invoke(messages)
        return {'messages':[message]}
    
    def take_action(self,state:AgentState):
        tool_calls=state['messages'][-1].tool_calls
        results=[]
        for t in tool_calls:
            print(f'Calling: {t}')
            result=self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t["id"],name=t["name"],content=str(result)))
        print("Back to the Model !!")
        return {"messages":results}
    

In [45]:
prompt = """You are a smart research assistant. Use the search engine to help answer user questions.
You are allowed to make multiple calls (either together or in sequence).
Only look up information when you are sure of what you want.
If you need to look up some information before asking a follow-up question, do so.
Be efficient and clear in your responses."""

tools=[tool]
abot=Agent(ChatGroq(model='llama3-8b-8192'),tools=tools,checkpointer=memory,system=prompt)

In [46]:
messages=[HumanMessage("What is the weather in varanasi ?")]
thread={"configurable":{"thread_id":"1"}}
for event in abot.graph.stream({"messages":messages},thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tgfx', 'function': {'arguments': '{"query":"weather in Varanasi"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 2063, 'total_tokens': 2137, 'completion_time': 0.061666667, 'prompt_time': 0.259154638, 'queue_time': -0.315787895, 'total_time': 0.320821305}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c2c0568a-f6fc-42b1-82bd-83535a1d9113-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Varanasi'}, 'id': 'call_tgfx', 'type': 'tool_call'}], usage_metadata={'input_tokens': 2063, 'output_tokens': 74, 'total_tokens': 2137})]}
()


In [47]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='What is the weather in varanasi ?', additional_kwargs={}, response_metadata={}, id='f3337f2b-3e60-465a-b9c6-2eb2d05a7585'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tgfx', 'function': {'arguments': '{"query":"weather in Varanasi"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 2063, 'total_tokens': 2137, 'completion_time': 0.061666667, 'prompt_time': 0.259154638, 'queue_time': -0.315787895, 'total_time': 0.320821305}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c2c0568a-f6fc-42b1-82bd-83535a1d9113-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Varanasi'}, 'id': 'call_tgfx', 'type': 'tool_call'}], usage_metadata={'input_tokens': 2063, 'output_tokens': 74, 'total_tokens': 2137})]}, next=('ac

In [48]:
abot.graph.get_state(thread).next

('action',)

In [49]:
for event in abot.graph.stream(None,thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Varanasi'}, 'id': 'call_tgfx', 'type': 'tool_call'}
Back to the Model !!
{'messages': [ToolMessage(content="[{'title': 'Weather in Varanasi in April 2025 (State of Uttar Pradesh)', 'url': 'https://world-weather.info/forecast/india/varanasi/april-2025/', 'content': 'Weather in Varanasi in April 2025. Varanasi Weather Forecast for April 2025 ... Monday, 21 April. +81°. Day. +100°. Clear sky. Tuesday, 22 April. +82°. Day. +99', 'score': 0.95407325}, {'title': 'Varanasi weather in April 2025 ⋆ Varanasi temperature in April ...', 'url': 'https://www.meteoprog.com/weather/Varanasi/month/april/', 'content': 'Varanasi (India) weather in April 2025 ☀ Accurate weather forecast for Varanasi in ... 21 Apr. +38°+26°. 22 Apr. +39°+25°. 23 Apr. +39°+26°. 24 Apr. +40°+25°. 25', 'score': 0.93161833}]", name='tavily_search_results_json', id='19bf356e-386a-4f1e-b76b-29089e932b85', tool_call_id='call_tgfx')]}
{'messages': [AIMes

In [50]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='What is the weather in varanasi ?', additional_kwargs={}, response_metadata={}, id='f3337f2b-3e60-465a-b9c6-2eb2d05a7585'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tgfx', 'function': {'arguments': '{"query":"weather in Varanasi"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 2063, 'total_tokens': 2137, 'completion_time': 0.061666667, 'prompt_time': 0.259154638, 'queue_time': -0.315787895, 'total_time': 0.320821305}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c2c0568a-f6fc-42b1-82bd-83535a1d9113-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Varanasi'}, 'id': 'call_tgfx', 'type': 'tool_call'}], usage_metadata={'input_tokens': 2063, 'output_tokens': 74, 'total_tokens': 2137}), ToolMessage

In [51]:
abot.graph.get_state(thread).next

()

In [52]:
messages=[HumanMessage("What is the weather in Varanasi ?")]
thread={"configurable":{"thread_id":"2"}}
for event in abot.graph.stream({"messages":messages},thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9h7b', 'function': {'arguments': '{"query":"weather in Varanasi"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 1021, 'total_tokens': 1102, 'completion_time': 0.0675, 'prompt_time': 0.141173133, 'queue_time': 0.23376384499999997, 'total_time': 0.208673133}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ee522ff3-7e73-4744-ba7e-904f649f981f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Varanasi'}, 'id': 'call_9h7b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1021, 'output_tokens': 81, 'total_tokens': 1102})]}
()


In [53]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='What is the weather in Varanasi ?', additional_kwargs={}, response_metadata={}, id='9eeae9f1-1d7d-4104-9356-6147a2e448c0'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9h7b', 'function': {'arguments': '{"query":"weather in Varanasi"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 1021, 'total_tokens': 1102, 'completion_time': 0.0675, 'prompt_time': 0.141173133, 'queue_time': 0.23376384499999997, 'total_time': 0.208673133}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ee522ff3-7e73-4744-ba7e-904f649f981f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Varanasi'}, 'id': 'call_9h7b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1021, 'output_tokens': 81, 'total_tokens': 1102})]}, next=('

In [54]:
abot.graph.get_state(thread).next

('action',)

In [55]:
current_values=abot.graph.get_state(thread)
current_values.values['messages'][-1]

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9h7b', 'function': {'arguments': '{"query":"weather in Varanasi"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 1021, 'total_tokens': 1102, 'completion_time': 0.0675, 'prompt_time': 0.141173133, 'queue_time': 0.23376384499999997, 'total_time': 0.208673133}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ee522ff3-7e73-4744-ba7e-904f649f981f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Varanasi'}, 'id': 'call_9h7b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1021, 'output_tokens': 81, 'total_tokens': 1102})

In [56]:
current_values.values['messages'][-1].tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'weather in Varanasi'},
  'id': 'call_9h7b',
  'type': 'tool_call'}]

In [57]:
current_values.values['messages'][-1].tool_calls[0]['args']['query']='weather in Noida'
current_values

StateSnapshot(values={'messages': [HumanMessage(content='What is the weather in Varanasi ?', additional_kwargs={}, response_metadata={}, id='9eeae9f1-1d7d-4104-9356-6147a2e448c0'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9h7b', 'function': {'arguments': '{"query":"weather in Varanasi"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 1021, 'total_tokens': 1102, 'completion_time': 0.0675, 'prompt_time': 0.141173133, 'queue_time': 0.23376384499999997, 'total_time': 0.208673133}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ee522ff3-7e73-4744-ba7e-904f649f981f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Noida'}, 'id': 'call_9h7b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1021, 'output_tokens': 81, 'total_tokens': 1102})]}, next=('act

In [58]:
abot.graph.update_state(thread,current_values.values)

{'configurable': {'thread_id': '2',
  'checkpoint_ns': '',
  'checkpoint_id': '1f01e791-b6e2-68d2-8002-42ad104b5c73'}}

In [59]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='What is the weather in Varanasi ?', additional_kwargs={}, response_metadata={}, id='9eeae9f1-1d7d-4104-9356-6147a2e448c0'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9h7b', 'function': {'arguments': '{"query":"weather in Varanasi"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 1021, 'total_tokens': 1102, 'completion_time': 0.0675, 'prompt_time': 0.141173133, 'queue_time': 0.23376384499999997, 'total_time': 0.208673133}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ee522ff3-7e73-4744-ba7e-904f649f981f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Noida'}, 'id': 'call_9h7b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1021, 'output_tokens': 81, 'total_tokens': 1102})]}, next=('act

In [60]:
for event in abot.graph.stream(None,thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Noida'}, 'id': 'call_9h7b', 'type': 'tool_call'}
Back to the Model !!
{'messages': [ToolMessage(content='[{\'title\': \'Weather in Noida\', \'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Noida\', \'region\': \'Uttar Pradesh\', \'country\': \'India\', \'lat\': 28.57, \'lon\': 77.32, \'tz_id\': \'Asia/Kolkata\', \'localtime_epoch\': 1745216597, \'localtime\': \'2025-04-21 11:53\'}, \'current\': {\'last_updated_epoch\': 1745216100, \'last_updated\': \'2025-04-21 11:45\', \'temp_c\': 36.3, \'temp_f\': 97.3, \'is_day\': 1, \'condition\': {\'text\': \'Mist\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/143.png\', \'code\': 1030}, \'wind_mph\': 6.5, \'wind_kph\': 10.4, \'wind_degree\': 215, \'wind_dir\': \'SW\', \'pressure_mb\': 1009.0, \'pressure_in\': 29.8, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 15, \'cloud\': 0, \'feelslike_c\': 34.8, \'feelslike_f\': 94.

In [61]:
for event in abot.graph.stream(None,thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Varanasi'}, 'id': 'call_vmde', 'type': 'tool_call'}
Back to the Model !!
{'messages': [ToolMessage(content="[{'title': 'Weather in Varanasi in April 2025 (State of Uttar Pradesh)', 'url': 'https://world-weather.info/forecast/india/varanasi/april-2025/', 'content': 'Weather in Varanasi in April 2025. Varanasi Weather Forecast for April 2025 ... Monday, 21 April. +81°. Day. +100°. Clear sky. Tuesday, 22 April. +82°. Day. +99', 'score': 0.95407325}, {'title': 'Varanasi weather in April 2025 ⋆ Varanasi temperature in April ...', 'url': 'https://www.meteoprog.com/weather/Varanasi/month/april/', 'content': 'Varanasi (India) weather in April 2025 ☀ Accurate weather forecast for Varanasi in ... 21 Apr. +38°+26°. 22 Apr. +39°+25°. 23 Apr. +39°+26°. 24 Apr. +40°+25°. 25', 'score': 0.93161833}]", name='tavily_search_results_json', id='6b1573a1-fe4f-483d-a173-205a4c2fafde', tool_call_id='call_vmde')]}
{'messages': [AIMes

In [62]:
states=[]
for state in abot.graph.get_state_history(thread):
    print(state)
    print("------")
    states.append(state)

StateSnapshot(values={'messages': [HumanMessage(content='What is the weather in Varanasi ?', additional_kwargs={}, response_metadata={}, id='9eeae9f1-1d7d-4104-9356-6147a2e448c0'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9h7b', 'function': {'arguments': '{"query":"weather in Varanasi"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 1021, 'total_tokens': 1102, 'completion_time': 0.0675, 'prompt_time': 0.141173133, 'queue_time': 0.23376384499999997, 'total_time': 0.208673133}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ee522ff3-7e73-4744-ba7e-904f649f981f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Noida'}, 'id': 'call_9h7b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1021, 'output_tokens': 81, 'total_tokens': 1102}), ToolMessage(

In [63]:
to_replay=states[-1]
to_replay

StateSnapshot(values={'messages': []}, next=('__start__',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f01e791-af3b-61c1-bfff-de0f0313c736'}}, metadata={'source': 'input', 'writes': {'__start__': {'messages': [HumanMessage(content='What is the weather in Varanasi ?', additional_kwargs={}, response_metadata={})]}}, 'step': -1, 'parents': {}, 'thread_id': '2'}, created_at='2025-04-21T06:23:14.053984+00:00', parent_config=None, tasks=(PregelTask(id='f9994fc0-da9f-d68e-7b39-bcddb0c3f23c', name='__start__', path=('__pregel_pull', '__start__'), error=None, interrupts=(), state=None, result={'messages': [HumanMessage(content='What is the weather in Varanasi ?', additional_kwargs={}, response_metadata={})]}),))

In [64]:
x=to_replay.config
for event in abot.graph.stream(None,x):
    for k,v in event.items():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zft9', 'function': {'arguments': '{"query":"weather in Varanasi"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 74, 'prompt_tokens': 2063, 'total_tokens': 2137, 'completion_time': 0.061666667, 'prompt_time': 0.25715902, 'queue_time': -0.34609064100000003, 'total_time': 0.318825687}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a044b772-9097-44f3-a5f9-2d20f868243e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Varanasi'}, 'id': 'call_zft9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 2063, 'output_tokens': 74, 'total_tokens': 2137})]}
()


In [65]:
for event in abot.graph.stream(None,thread):
    for k,v in event.items():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Varanasi'}, 'id': 'call_zft9', 'type': 'tool_call'}
Back to the Model !!
{'messages': [ToolMessage(content="[{'title': 'Weather in Varanasi in April 2025 (State of Uttar Pradesh)', 'url': 'https://world-weather.info/forecast/india/varanasi/april-2025/', 'content': 'Weather in Varanasi in April 2025. Varanasi Weather Forecast for April 2025 ... Monday, 21 April. +81°. Day. +100°. Clear sky. Tuesday, 22 April. +82°. Day. +99', 'score': 0.95407325}, {'title': 'Varanasi weather in April 2025 ⋆ Varanasi temperature in April ...', 'url': 'https://www.meteoprog.com/weather/Varanasi/month/april/', 'content': 'Varanasi (India) weather in April 2025 ☀ Accurate weather forecast for Varanasi in ... 21 Apr. +38°+26°. 22 Apr. +39°+25°. 23 Apr. +39°+26°. 24 Apr. +40°+25°. 25', 'score': 0.93161833}]", name='tavily_search_results_json', id='3da3793c-5462-4ec7-af38-3c7a00ed1209', tool_call_id='call_zft9')]}
{'messages': [AIMes

In [66]:
to_replay.values['messages'][-1].tool_calls[0]['args']['query']='weather in Varanasi, NDTV'

IndexError: list index out of range